In [26]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [27]:
batch_size = 2  # Puedes ajustar este valor según la disponibilidad de memoria
lr = 1e-5  # 0.0001
epochs = 40
height = 960
width = 540

In [28]:
dataset_path = os.path.join("dataset", "non-aug")
files_dir = os.path.join("Colab Notebooks", "files", "non-aug")
model_file = os.path.join(files_dir, "unet-non-aug.h5")
log_file = os.path.join(files_dir, "log-non-aug.csv")

In [29]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [30]:
create_dir(files_dir)

In [31]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [32]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [33]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding="same")(inputs)
    x = tf.image.resize(x, [skip.shape[1], skip.shape[2]])  # Asegúrate de que las dimensiones coinciden
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

In [34]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    """Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """Bridge"""
    b1 = conv_block(p4, 1024)

    """Decoder"""
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

In [1]:
def load_data(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*")))
    train_y = sorted(glob(os.path.join(path, "train", "masks", "*")))
    valid_x = sorted(glob(os.path.join(path, "valid", "images", "*")))
    valid_y = sorted(glob(os.path.join(path, "valid", "masks", "*")))

    return (train_x, train_y), (valid_x, valid_y)

In [36]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (width, height))
    x = x / 255.0
    return x

In [37]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (width, height))
    x = x / 255.0
    x = np.expand_dims(x, axis=-1)
    return x

In [38]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return tf.convert_to_tensor(x, dtype=tf.float32), tf.convert_to_tensor(y, dtype=tf.float32)

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])
    return x, y

In [39]:
def tf_dataset(x, y, batch=batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [40]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

Train: 455 - 455
Valid: 455 - 455


In [41]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

In [42]:
for x, y in valid_dataset:
    print(x.shape, y.shape)

(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 3) (1, 960, 540, 1)
(1, 960, 540, 

In [43]:
input_shape = (height, width, 3)
model = build_unet(input_shape)

In [44]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 960, 540, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 960, 540, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 960, 540, 64  256        ['conv2d_19[0][0]']              
 ormalization)                  )                                                              

In [45]:
opt = tf.keras.optimizers.Adam(learning_rate=lr, clipnorm=1.0)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["acc"])

In [46]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
]

In [47]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/40
455/455 [==============================] - ETA: 0s - loss: 0.2672 - acc: 0.9253
Epoch 1: val_loss improved from inf to 0.52245, saving model to Colab Notebooks\files\non-aug\unet-non-aug.h5
455/455 [==============================] - 301s 654ms/step - loss: 0.2672 - acc: 0.9253 - val_loss: 0.5225 - val_acc: 0.7645 - lr: 1.0000e-05
Epoch 2/40
455/455 [==============================] - ETA: 0s - loss: 0.1568 - acc: 0.9588
Epoch 2: val_loss improved from 0.52245 to 0.45351, saving model to Colab Notebooks\files\non-aug\unet-non-aug.h5
455/455 [==============================] - 465s 1s/step - loss: 0.1568 - acc: 0.9588 - val_loss: 0.4535 - val_acc: 0.8183 - lr: 1.0000e-05
Epoch 3/40
455/455 [==============================] - ETA: 0s - loss: 0.1124 - acc: 0.9705
Epoch 3: val_loss did not improve from 0.45351
455/455 [==============================] - 252s 553ms/step - loss: 0.1124 - acc: 0.9705 - val_loss: 0.6216 - val_acc: 0.7439 - lr: 1.0000e-05
Epoch 4/40
455/455 [=============